# Validator Agent 테스트

검증 에이전트 (Level 3) 테스트 노트북

In [ ]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

In [ ]:
def create_base_state():
    return {"content": "테스트 스토리", "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/api/callback",
            "extracted_characters": [], "extracted_events": [],
            "validation_result": {}, "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

In [ ]:
from app.agents.validation.validator import validator_node

# 정상 케이스 (approve)
async def test_validator_approve():
    print("✅ Validator (approve) 테스트...")
    state = create_base_state()
    state["extracted_characters"] = [{"name": "서진"}, {"name": "하나"}]
    state["extracted_events"] = [{"event_id": "E001"}]
    return await validator_node(state)

result = run_async(test_validator_approve())
print(json.dumps(result, ensure_ascii=False, indent=2))

In [ ]:
# 경고 케이스 (human_review)
async def test_validator_review():
    print("⚠️ Validator (human_review) 테스트...")
    state = create_base_state()
    state["extracted_characters"] = []  # 캐릭터 없음 (-20)
    state["extracted_events"] = [{"event_id": "E001"}]
    state["errors"] = ["Minor error"]  # 에러 1개 (-5)
    return await validator_node(state)

review_result = run_async(test_validator_review())
print(json.dumps(review_result, ensure_ascii=False, indent=2))

In [ ]:
# 실패 케이스 (retry_extraction)
async def test_validator_retry():
    print("❌ Validator (retry_extraction) 테스트...")
    state = create_base_state()
    state["extracted_characters"] = []  # -20
    state["extracted_events"] = []  # -10
    state["errors"] = ["Error 1", "Error 2", "Error 3", "Error 4", "Error 5"]  # -25
    return await validator_node(state)

retry_result = run_async(test_validator_retry())
print(json.dumps(retry_result, ensure_ascii=False, indent=2))